In [1]:
import os
import glob
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms, models
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.model_selection import KFold



In [2]:


# Define constants
SERIES_DESCRIPTIONS = ['Sagittal T1', 'Sagittal T2_STIR', 'Axial T2']
CONDITIONS = [
    'spinal_canal_stenosis', 
    'left_neural_foraminal_narrowing', 
    'right_neural_foraminal_narrowing',
    'left_subarticular_stenosis',
    'right_subarticular_stenosis'
]
LEVELS = [
    'l1_l2',
    'l2_l3',
    'l3_l4',
    'l4_l5',
    'l5_s1',
]
LABELS = [f'{condition}_{level}' for condition in CONDITIONS for level in LEVELS]

# Set the root directory for your Kaggle files
rd = './kaggle-files'

# Load the main CSV file
df = pd.read_csv(f'{rd}/train.csv')

df = df.fillna(-100)  # Use -100 to indicate missing labels

# Map the labels to integers for multi-class classification
label2id = {'Normal/Mild': 0, 'Moderate': 1, 'Severe': 2}
df.replace(label2id, inplace=True)

# Load the coordinates data
coordinates_df = pd.read_csv(f'{rd}/dfc_updated.csv')
# Keep only rows where 'slice_number' is not NaN
coordinates_df = coordinates_df.dropna(subset=['slice_number'])
coordinates_df['slice_number'] = coordinates_df['slice_number'].astype(int)

# Load the series descriptions
series_description_df = pd.read_csv(f'{rd}/train_series_descriptions.csv')
series_description_df['series_description'] = series_description_df['series_description'].str.replace('T2/STIR', 'T2_STIR')

In [3]:


class LumbarSpineDataset(Dataset):
    def __init__(self, df, coordinates_df, series_description_df, root_dir, transform=None):
        self.df = df
        self.coordinates_df = coordinates_df
        self.series_description_df = series_description_df
        self.root_dir = root_dir  # The root directory where images are stored
        self.transform = transform

        # Get the list of study_ids
        self.study_ids = self.df['study_id'].unique()

        # List of label columns
        self.label_columns = [col for col in df.columns if col != 'study_id']

        # Prepare a mapping for images and annotations
        self.study_image_paths = self._prepare_image_paths()

        # Create a mapping from study_id to labels
        self.labels_dict = self._prepare_labels()

    def _prepare_image_paths(self):
        study_image_paths = {}
        for study_id in self.study_ids:
            study_image_paths[study_id] = {}
            for series_description in SERIES_DESCRIPTIONS:
                series_description_clean = series_description.replace('/', '_')
                image_dir = os.path.join(self.root_dir, 'cvt_png', str(study_id), series_description_clean)
                if os.path.exists(image_dir):
                    # Get all images in the directory
                    image_paths = sorted(glob.glob(os.path.join(image_dir, '*.png')))
                    study_image_paths[study_id][series_description] = image_paths
                else:
                    # Handle missing series
                    study_image_paths[study_id][series_description] = []
        return study_image_paths

    def _prepare_labels(self):
        labels_dict = {}
        for idx, row in self.df.iterrows():
            study_id = row['study_id']
            labels = []
            for col in self.label_columns:
                label = row[col]
                if pd.isnull(label) or label == -100:
                    label = -100  # Use -100 for missing labels (ignore_index)
                else:
                    label = int(label)
                labels.append(label)
            labels_dict[study_id] = labels
        return labels_dict

    def __len__(self):
        return len(self.study_ids)

    def __getitem__(self, idx):
        study_id = self.study_ids[idx]
        images = {}
        annotations = {}

        # Load images for each series description
        for series_description in SERIES_DESCRIPTIONS:
            image_paths = self.study_image_paths[study_id][series_description]
            series_images = []
            for img_path in image_paths:
                img = Image.open(img_path).convert('L')  # Convert to grayscale
                if self.transform:
                    img = self.transform(img)  # Shape: [1, H, W]
                    img = img.squeeze(0)  # Remove the channel dimension, resulting in [H, W]
                series_images.append(img)
            if series_images:
                series_tensor = torch.stack(series_images, dim=0)  # Shape: [num_slices, H, W]
            else:
                series_tensor = torch.zeros((1, 512, 512))  # Placeholder tensor
            images[series_description] = series_tensor  # Shape: [num_slices, H, W]

        # Get labels for the study_id
        labels = self.labels_dict[study_id]
        labels_tensor = torch.tensor(labels, dtype=torch.long)  # Use long dtype for CrossEntropyLoss

        # Generate attention masks, default to zeros if no annotations
        attention_masks = {}
        for series_description in SERIES_DESCRIPTIONS:
            series_tensor = images[series_description]
            num_slices = series_tensor.shape[0]
            masks = []
            for slice_idx in range(num_slices):
                image_shape = series_tensor[slice_idx].shape  # Get (H, W)
                mask = torch.zeros(image_shape, dtype=torch.float32)  # Default to zero mask
                # If annotations exist, generate the attention mask
                study_annotations = self.coordinates_df[self.coordinates_df['study_id'] == study_id]
                for _, row in study_annotations.iterrows():
                    if row['series_description'] == series_description:
                        x_pixel = int(row['x_scaled'] * image_shape[1])
                        y_pixel = int(row['y_scaled'] * image_shape[0])
                        sigma = 5  # Adjust sigma
                        y_grid, x_grid = torch.meshgrid(
                            torch.arange(image_shape[0], dtype=torch.float32),
                            torch.arange(image_shape[1], dtype=torch.float32),
                            indexing='ij'
                        )
                        gauss = torch.exp(-((x_grid - x_pixel) ** 2 + (y_grid - y_pixel) ** 2) / (2 * sigma ** 2))
                        mask = torch.maximum(mask, gauss)
                masks.append(mask)
            attention_masks[series_description] = torch.stack(masks, dim=0)  # Shape: [num_slices, H, W]

        sample = {
            'study_id': study_id,
            'images': images,
            'labels': labels_tensor,
            'attention_masks': attention_masks
        }

        return sample

In [4]:


# Define any transformations if needed
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Adjust mean and std if necessary
])

# Instantiate the dataset
train_dataset = LumbarSpineDataset(
    df=df,
    coordinates_df=coordinates_df,
    series_description_df=series_description_df,
    root_dir='./rsna_output',  # Adjust the path as needed
    transform=transform
)


In [5]:
def resample_slices(image_tensor, target_slices=10):
    # Ensure the image tensor has at least 3 dimensions
    if image_tensor.dim() == 2:
        image_tensor = image_tensor.unsqueeze(0)  # Add slice dimension
    current_slices = image_tensor.shape[0]
    if current_slices == target_slices:
        return image_tensor  # No need to resample
    if current_slices > target_slices:
        indices = torch.linspace(0, current_slices - 1, target_slices).long()
        return image_tensor[indices]
    # If fewer slices, upsample
    image_tensor = image_tensor.unsqueeze(0).unsqueeze(0)  # Shape: [1, 1, num_slices, H, W]
    image_tensor_resized = F.interpolate(
        image_tensor,
        size=(target_slices, image_tensor.shape[3], image_tensor.shape[4]),
        mode='trilinear',
        align_corners=False
    )
    image_tensor_resized = image_tensor_resized.squeeze(0).squeeze(0)  # Shape: [target_slices, H, W]
    return image_tensor_resized

# Early Stopping class
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.counter = 0
        self.early_stop = False
        self.best_loss = float('inf')

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Save model when validation loss decreases.'''
        self.best_loss = val_loss
        torch.save(model.state_dict(), 'checkpoint.pth')

# Define the ResNet feature extractor
class ResNetFeatureExtractor(nn.Module):
    def __init__(self, in_channels=10):
        super(ResNetFeatureExtractor, self).__init__()
        resnet = models.resnet18(pretrained=True)

        # Modify the first convolutional layer to accept in_channels
        resnet.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Extract layers up to layer4 (exclude avgpool and fc layers)
        self.features = nn.Sequential(
            resnet.conv1,
            resnet.bn1,
            resnet.relu,
            resnet.maxpool,
            resnet.layer1,
            resnet.layer2,
            resnet.layer3,
            resnet.layer4
        )

    def forward(self, x):
        x = self.features(x)
        return x

# Define the main model
class MultiSeriesSpineModel(nn.Module):
    def __init__(self, num_conditions=25, num_classes=3):
        super(MultiSeriesSpineModel, self).__init__()
        self.num_conditions = num_conditions
        self.num_classes = num_classes

        # Feature extractors for each MRI series
        self.cnn_sagittal_t1 = ResNetFeatureExtractor(in_channels=10)
        self.cnn_sagittal_t2_stir = ResNetFeatureExtractor(in_channels=10)
        self.cnn_axial_t2 = ResNetFeatureExtractor(in_channels=10)

        # Define attention layers for each series
        self.attention_sagittal_t1 = nn.Sequential(
            nn.Conv2d(512, 1, kernel_size=1),
            nn.Sigmoid()
        )
        self.attention_sagittal_t2_stir = nn.Sequential(
            nn.Conv2d(512, 1, kernel_size=1),
            nn.Sigmoid()
        )
        self.attention_axial_t2 = nn.Sequential(
            nn.Conv2d(512, 1, kernel_size=1),
            nn.Sigmoid()
        )

        # Define the final classification layers
        combined_feature_size = 512 * 3  # Since we're concatenating features from three models

        self.fc1 = nn.Linear(combined_feature_size, 512)
        self.fc2 = nn.Linear(512, num_conditions * num_classes)  # Output layer

    def forward(self, sagittal_t1, sagittal_t2_stir, axial_t2):
        # The tensors are of shape [batch_size, in_channels, H, W]
        features_sagittal_t1 = self.cnn_sagittal_t1(sagittal_t1)  # Shape: [batch_size, 512, H, W]
        features_sagittal_t2_stir = self.cnn_sagittal_t2_stir(sagittal_t2_stir)
        features_axial_t2 = self.cnn_axial_t2(axial_t2)

        # Generate attention maps (learned by the model)
        attention_map_t1 = self.attention_sagittal_t1(features_sagittal_t1)  # Shape: [batch_size, 1, H, W]
        attention_map_t2_stir = self.attention_sagittal_t2_stir(features_sagittal_t2_stir)
        attention_map_axial = self.attention_axial_t2(features_axial_t2)

        # Apply attention
        attended_features_t1 = features_sagittal_t1 * attention_map_t1  # Element-wise multiplication
        attended_features_t2_stir = features_sagittal_t2_stir * attention_map_t2_stir
        attended_features_axial = features_axial_t2 * attention_map_axial

        # Global average pooling
        features_sagittal_t1 = F.adaptive_avg_pool2d(attended_features_t1, (1, 1)).view(attended_features_t1.size(0), -1)
        features_sagittal_t2_stir = F.adaptive_avg_pool2d(attended_features_t2_stir, (1, 1)).view(attended_features_t2_stir.size(0), -1)
        features_axial_t2 = F.adaptive_avg_pool2d(attended_features_axial, (1, 1)).view(attended_features_axial.size(0), -1)

        # Concatenate features
        combined_features = torch.cat([features_sagittal_t1, features_sagittal_t2_stir, features_axial_t2], dim=1)

        # Pass through final classification layers
        x = F.relu(self.fc1(combined_features))
        x = self.fc2(x)  # Shape: [batch_size, num_conditions * num_classes]
        x = x.view(-1, self.num_conditions, self.num_classes)  # Reshape to [batch_size, num_conditions, num_classes]

        return x, [attention_map_t1, attention_map_t2_stir, attention_map_axial]

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def custom_loss(outputs, labels, any_severe_scalar=0.5):
    batch_size = outputs.shape[0]
    num_conditions = outputs.shape[1]
    num_classes = outputs.shape[2]

    # Map conditions to label indices
    condition_to_indices = {}
    for condition in CONDITIONS:
        condition_to_indices[condition] = [i for i, label in enumerate(LABELS) if condition in label]

    condition_losses = []
    condition_weights = []

    classification_criterion = nn.CrossEntropyLoss(ignore_index=-100, reduction='mean')

    for condition in CONDITIONS:
        indices = condition_to_indices[condition]
        outputs_condition = outputs[:, indices, :]  # Shape: [batch_size, num_labels_in_condition, num_classes]
        labels_condition = labels[:, indices]  # Shape: [batch_size, num_labels_in_condition]

        # Reshape to compute loss
        outputs_condition = outputs_condition.reshape(-1, num_classes)
        labels_condition = labels_condition.reshape(-1)

        # Exclude samples with ignore_index
        valid_mask = labels_condition != -100
        if valid_mask.sum() > 0:
            outputs_valid = outputs_condition[valid_mask]
            labels_valid = labels_condition[valid_mask]

            loss_condition = classification_criterion(outputs_valid, labels_valid)
            condition_losses.append(loss_condition)
            condition_weights.append(1.0)
        else:
            pass  # Skip if all labels are missing

    # Compute 'any severe' loss for 'spinal_canal_stenosis'
    spinal_indices = condition_to_indices['spinal_canal_stenosis']
    outputs_spinal = outputs[:, spinal_indices, :]  # Shape: [batch_size, num_spinal_labels, num_classes]
    labels_spinal = labels[:, spinal_indices]  # Shape: [batch_size, num_spinal_labels]

    # Get severe class outputs (class index 2)
    severe_preds_spinal = outputs_spinal[:, :, 2]  # Shape: [batch_size, num_spinal_labels]
    severe_labels_spinal = (labels_spinal == 2).float()  # Shape: [batch_size, num_spinal_labels]

    # Ignore labels with -100
    valid_mask = labels_spinal != -100
    severe_preds_spinal = severe_preds_spinal * valid_mask.float()
    severe_labels_spinal = severe_labels_spinal * valid_mask.float()

    # For each sample in batch, get max severe label and prediction
    any_severe_label = torch.max(severe_labels_spinal, dim=1)[0]  # Shape: [batch_size]
    any_severe_pred = torch.max(severe_preds_spinal, dim=1)[0]  # Shape: [batch_size]

    # Compute binary cross-entropy loss
    any_severe_loss = F.binary_cross_entropy_with_logits(any_severe_pred, any_severe_label)

    # Append to losses
    condition_losses.append(any_severe_loss)
    condition_weights.append(any_severe_scalar)

    # Compute weighted average of losses
    total_loss = 0.0
    total_weight = sum(condition_weights)
    for loss, weight in zip(condition_losses, condition_weights):
        total_loss += loss * weight
    total_loss = total_loss / total_weight

    return total_loss





In [6]:

def custom_collate_fn(batch):
    collated_batch = {}
    # Handle 'study_id' separately
    collated_batch['study_id'] = [item['study_id'] for item in batch]
    # Handle 'labels'
    labels_list = []
    for item in batch:
        labels = item['labels']
        if not isinstance(labels, torch.Tensor):
            labels = torch.tensor(labels, dtype=torch.long)
        if labels.dim() == 0:
            labels = labels.unsqueeze(0)
        labels_list.append(labels)
    collated_batch['labels'] = torch.stack(labels_list)
    # Handle 'images' and 'attention_masks'
    for key in ['images', 'attention_masks']:
        collated_batch[key] = {}
        sub_keys = batch[0][key].keys()
        for sub_key in sub_keys:
            items_list = []
            for item in batch:
                data = item[key][sub_key]
                if not isinstance(data, torch.Tensor):
                    data = torch.tensor(data)
                items_list.append(data)
            collated_batch[key][sub_key] = torch.stack(items_list)
    return collated_batch


In [7]:
# Adjusted training loop with try-except blocks
def train_k_fold_with_custom_loss(train_dataset, k_folds=5, num_epochs=10, any_severe_scalar=0.5, lambda_attention=0.1):
    kfold = KFold(n_splits=k_folds, shuffle=True)

    for fold, (train_ids, val_ids) in enumerate(kfold.split(train_dataset)):
        print(f'Fold {fold+1}/{k_folds}')

        # Create data loaders for this fold with custom collate function
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)

        train_loader = DataLoader(
            train_dataset,
            batch_size=1,
            sampler=train_subsampler,
            num_workers=8,
            pin_memory=True,
            collate_fn=custom_collate_fn
        )

        val_loader = DataLoader(
            train_dataset,
            batch_size=1,
            sampler=val_subsampler,
            num_workers=8,
            pin_memory=True,
            collate_fn=custom_collate_fn
        )

        # Initialize the model and move it to the correct device
        model = MultiSeriesSpineModel(num_conditions=len(train_dataset.label_columns), num_classes=3)
        model = model.to(device)

        # Initialize optimizer and loss functions
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

        early_stopping = EarlyStopping(patience=3)

        for epoch in range(num_epochs):
            model.train()
            total_train_loss = 0.0

            # Training loop
            for batch in tqdm(train_loader, desc=f"Fold {fold+1} Epoch {epoch+1}/{num_epochs}", unit="batch"):
                try:
                    # Extract images, labels, and attention masks
                    images = batch['images']
                    labels = batch['labels'].to(device)  # Shape: [batch_size, num_conditions]
                    attention_masks = batch['attention_masks']

                    # Process each series
                    sagittal_t1 = images['Sagittal T1']  # Shape: [batch_size, num_slices, H, W]
                    sagittal_t2_stir = images['Sagittal T2_STIR']
                    axial_t2 = images['Axial T2']

                    # Resample slices
                    sagittal_t1 = [resample_slices(img.squeeze(0), target_slices=10) for img in sagittal_t1]
                    sagittal_t2_stir = [resample_slices(img.squeeze(0), target_slices=10) for img in sagittal_t2_stir]
                    axial_t2 = [resample_slices(img.squeeze(0), target_slices=10) for img in axial_t2]

                    # Check shapes before stacking
                    for img in sagittal_t1:
                        if img.shape != (10, 512, 512):
                            print(f"Invalid image shape in sagittal_t1: {img.shape}")
                            raise ValueError("Invalid image shape in sagittal_t1")
                    for img in sagittal_t2_stir:
                        if img.shape != (10, 512, 512):
                            print(f"Invalid image shape in sagittal_t2_stir: {img.shape}")
                            raise ValueError("Invalid image shape in sagittal_t2_stir")
                    for img in axial_t2:
                        if img.shape != (10, 512, 512):
                            print(f"Invalid image shape in axial_t2: {img.shape}")
                            raise ValueError("Invalid image shape in axial_t2")

                    # Stack slices into the channel dimension
                    sagittal_t1 = torch.stack([img.reshape(-1, 512, 512) for img in sagittal_t1]).to(device)
                    sagittal_t2_stir = torch.stack([img.reshape(-1, 512, 512) for img in sagittal_t2_stir]).to(device)
                    axial_t2 = torch.stack([img.reshape(-1, 512, 512) for img in axial_t2]).to(device)

                    # Move attention masks to the same device
                    mask_t1 = attention_masks['Sagittal T1'].to(device)  # Shape: [batch_size, num_slices, H, W]
                    mask_t2_stir = attention_masks['Sagittal T2_STIR'].to(device)
                    mask_axial = attention_masks['Axial T2'].to(device)

                    # Combine masks across slices (max over slices)
                    gt_mask_t1 = torch.max(mask_t1, dim=1)[0].unsqueeze(1)  # Shape: [batch_size, 1, H, W]
                    gt_mask_t2_stir = torch.max(mask_t2_stir, dim=1)[0].unsqueeze(1)
                    gt_mask_axial = torch.max(mask_axial, dim=1)[0].unsqueeze(1)

                    # Forward pass
                    outputs, attention_maps = model(sagittal_t1, sagittal_t2_stir, axial_t2)

                    # Compute the custom loss
                    classification_loss = custom_loss(outputs, labels, any_severe_scalar=any_severe_scalar)

                    # Compute attention loss
                    attention_loss = 0.0
                    attention_criterion = nn.MSELoss()
                    for attention_map, gt_mask in zip(attention_maps, [gt_mask_t1, gt_mask_t2_stir, gt_mask_axial]):
                        # Upsample the attention map to match the ground truth mask size
                        attention_map_upsampled = F.interpolate(attention_map, size=gt_mask.shape[2:], mode='bilinear', align_corners=False)
                        attention_loss += attention_criterion(attention_map_upsampled, gt_mask)

                    # Total loss
                    total_loss = classification_loss + lambda_attention * attention_loss

                    # Zero gradients
                    optimizer.zero_grad()

                    # Backpropagation and optimization
                    total_loss.backward()
                    optimizer.step()

                    total_train_loss += total_loss.item()

                except Exception as e:
                    print(f"Error processing batch during training: {e}")
                    continue  # Skip this batch

            avg_train_loss = total_train_loss / len(train_loader)

            # Validation loop
            model.eval()
            total_val_loss = 0.0
            with torch.no_grad():
                for batch in val_loader:
                    try:
                        images = batch['images']
                        labels = batch['labels'].to(device)
                        attention_masks = batch['attention_masks']
                
                        # Preprocess images
                        sagittal_t1 = [resample_slices(img.squeeze(0), target_slices=10) for img in images['Sagittal T1']]
                        sagittal_t2_stir = [resample_slices(img.squeeze(0), target_slices=10) for img in images['Sagittal T2_STIR']]
                        axial_t2 = [resample_slices(img.squeeze(0), target_slices=10) for img in images['Axial T2']]

                        # Check shapes before stacking
                        for img in sagittal_t1:
                            if img.shape != (10, 512, 512):
                                print(f"Invalid image shape in sagittal_t1 (validation): {img.shape}")
                                raise ValueError("Invalid image shape in sagittal_t1 (validation)")
                        for img in sagittal_t2_stir:
                            if img.shape != (10, 512, 512):
                                print(f"Invalid image shape in sagittal_t2_stir (validation): {img.shape}")
                                raise ValueError("Invalid image shape in sagittal_t2_stir (validation)")
                        for img in axial_t2:
                            if img.shape != (10, 512, 512):
                                print(f"Invalid image shape in axial_t2 (validation): {img.shape}")
                                raise ValueError("Invalid image shape in axial_t2 (validation)")

                        sagittal_t1 = torch.stack([img.reshape(-1, 512, 512) for img in sagittal_t1]).to(device)
                        sagittal_t2_stir = torch.stack([img.reshape(-1, 512, 512) for img in sagittal_t2_stir]).to(device)
                        axial_t2 = torch.stack([img.reshape(-1, 512, 512) for img in axial_t2]).to(device)
                
                        mask_t1 = attention_masks['Sagittal T1'].to(device)  # Shape: [batch_size, num_slices, H, W]
                        mask_t2_stir = attention_masks['Sagittal T2_STIR'].to(device)
                        mask_axial = attention_masks['Axial T2'].to(device)
                
                        # Combine masks across slices (max over slices)
                        gt_mask_t1 = torch.max(mask_t1, dim=1)[0].unsqueeze(1)  # Shape: [batch_size, 1, H, W]
                        gt_mask_t2_stir = torch.max(mask_t2_stir, dim=1)[0].unsqueeze(1)
                        gt_mask_axial = torch.max(mask_axial, dim=1)[0].unsqueeze(1)
                
                        outputs, attention_maps = model(sagittal_t1, sagittal_t2_stir, axial_t2)
                
                        classification_loss = custom_loss(outputs, labels, any_severe_scalar=any_severe_scalar)
                
                        attention_loss = 0.0
                        attention_criterion = nn.MSELoss()
                        for attention_map, gt_mask in zip(attention_maps, [gt_mask_t1, gt_mask_t2_stir, gt_mask_axial]):
                            # Upsample the attention map to match the ground truth mask size
                            attention_map_upsampled = F.interpolate(
                                attention_map,
                                size=gt_mask.shape[2:],  # This will be [H, W]
                                mode='bilinear',
                                align_corners=False
                            )
                            attention_loss += attention_criterion(attention_map_upsampled, gt_mask)
                
                        total_loss = classification_loss + lambda_attention * attention_loss
                
                        total_val_loss += total_loss.item()

                    except Exception as e:
                        print(f"Error processing batch during validation: {e}")
                        continue  # Skip this batch
                
                avg_val_loss = total_val_loss / len(val_loader) if len(val_loader) > 0 else float('inf')
            
                print(f"Fold {fold+1} Epoch [{epoch+1}/{num_epochs}] Avg Train Loss: {avg_train_loss:.4f}, Avg Val Loss: {avg_val_loss:.4f}")
            
                # Early stopping based on validation loss
                early_stopping(avg_val_loss, model)
                if early_stopping.early_stop:
                    print(f"Early stopping triggered for Fold {fold+1}!")
                    break


        # Save the model for this fold
        torch.save(model.state_dict(), f'model_fold_{fold+1}.pth')
        print(f"Completed Fold {fold+1}/{k_folds}. Model saved to model_fold_{fold+1}.pth.")


In [ ]:
# Example usage

n_folds = 5

train_k_fold_with_custom_loss(train_dataset, k_folds=n_folds, num_epochs=5)


Fold 1/5


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can als

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EnsembleModel(nn.Module):
    def __init__(self, model_class, model_paths, device):
        super(EnsembleModel, self).__init__()
        self.models = nn.ModuleList()
        for path in model_paths:
            model = model_class()
            model.load_state_dict(torch.load(path, map_location=device))
            model.to(device)
            model.eval()  # Set model to evaluation mode
            self.models.append(model)
        self.device = device

    def forward(self, sagittal_t1, sagittal_t2_stir, axial_t2):
        outputs_list = []
        for model in self.models:
            outputs, _ = model(sagittal_t1, sagittal_t2_stir, axial_t2)
            outputs_list.append(outputs)
        # Stack outputs and take mean over the ensemble dimension
        outputs = torch.stack(outputs_list, dim=0)
        avg_outputs = torch.mean(outputs, dim=0)
        return avg_outputs


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
k_folds = n_folds
model_paths = [f'model_fold_{i+1}.pth' for i in range(k_folds)]

ensemble_model = EnsembleModel(
    model_class=lambda: MultiSeriesSpineModel(num_conditions=len(LABELS), num_classes=3),
    model_paths=model_paths,
    device=device
)

ensemble_model.to(device)
ensemble_model.eval()


EnsembleModel(
  (models): ModuleList(
    (0-4): 5 x MultiSeriesSpineModel(
      (cnn_sagittal_t1): ResNetFeatureExtractor(
        (features): Sequential(
          (0): Conv2d(10, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (4): Sequential(
            (0): BasicBlock(
              (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (relu): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (1)

In [11]:
torch.save(ensemble_model.state_dict(), 'ensemble_model.pth')
